In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
block_size = 3

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] 

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])    
Xdev, Ydev = build_dataset(words[n1:n2])   
Xte,  Yte  = build_dataset(words[n2:])     

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 
n_hidden = 64 

g = torch.Generator().manual_seed(2147483647) 
C  = torch.randn((vocab_size, n_embd),            generator=g)

W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 

W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1

bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1



parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
  p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size 

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] 

In [ ]:

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
hprebn = embcat @ W1 + b1 
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) 
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

h = torch.tanh(hpreact) 
logits = h @ W2 + b2 
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes 
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, 
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3204, grad_fn=<NegBackward0>)

In [34]:
hprebn.shape, embcat.shape,W1.shape , b1.shape 

(torch.Size([32, 64]),
 torch.Size([32, 30]),
 torch.Size([30, 64]),
 torch.Size([64]))

In [ ]:
dlogprobs=torch.zeros_like(logprobs)
dlogprobs[range(n),Yb]=-1/n
dprobs=1/probs.data * dlogprobs
dcounts_sum_inv= (counts * dprobs).sum(1,keepdim=True)
dcounts_sum=-(counts_sum**-2) * dcounts_sum_inv
dcounts=counts_sum_inv * dprobs +  dcounts_sum
dnorm_logits=counts *dcounts
dlogit_maxs=-dnorm_logits.sum(1,keepdim=True)
dlogits= dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices,num_classes=logits.shape[1]) * dlogit_maxs
dh = dlogits @ W2.T
dW2=  h.T @ dlogits
db2= dlogits.sum(0,keepdim=True)
dhpreact=(1-h**2) * dh
dbngain = (bnraw * dhpreact).sum(0,keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0,keepdim=True)
dbnvar_inv = (bndiff * dbnraw).sum(0,keepdim=True)
dbndiff = (bnvar_inv * dbnraw)
dbnvar=-0.5 * (bnvar+1e-5)**-1.5 * dbnvar_inv
dbndiff2 = torch.ones_like(bndiff2) * 1/(n-1) * dbnvar
dbndiff += 2*bndiff * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = - dbndiff.sum(0,keepdim=True)
dhprebn += 1/n * dbnmeani
dembcat= dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1=dhprebn.sum(0)
print(embcat.shape[1]/n_embd)
demb= dembcat.reshape(1,embcat.shape[1]/n_embd,n_embd)

print(demb.shape,emb.shape)
#Checking
cmp('logprobs',dlogprobs,logprobs)
cmp('probs',dprobs,probs)
cmp('counts sum invert',dcounts_sum_inv,counts_sum_inv)
cmp('counts sum',dcounts_sum,counts_sum)
cmp('counts',dcounts,counts)
cmp('norm logits',dnorm_logits,norm_logits)
cmp('dlogits max',dlogit_maxs,logit_maxes)
cmp('b2',db2,b2)
cmp('h',dh,h)
cmp('W2',dW2,W2)
cmp('hpreact',dhpreact,hpreact)
cmp('bnbias',dbnbias,bnbias)
cmp('bngain',dbngain,bngain)
cmp('bnraw',dbnraw,bnraw)
cmp('bnvar_inv',dbnvar_inv,bnvar_inv)
cmp('bnvar',dbnvar,bnvar)
cmp('bndiff2',dbndiff2,bndiff2)
cmp('bndiff',dbndiff,bndiff)
cmp('hprebn',dhprebn,hprebn)
cmp('bnmeani',dbnmeani,bnmeani)
cmp('embcat',dembcat,embcat)
cmp('W1',dW1,W1)
cmp('b1',db1,b1)
cmp('emb',demb,emb)




TypeError: reshape(): argument 'shape' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got float"